In [ ]:

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [ ]:
url = 'https://www.dbpia.co.kr/search/topSearch?searchOption=all&query=%EB%94%A5%EB%9F%AC%EB%8B%9D#a'

res = []

driver = webdriver.Chrome() # 크롬 드라이버 생성
driver.get(url) # 사이트 접속하기

checkbox = driver.find_element(By.ID, 'domesticRecord_064001') # KCI 등재 체크박스

if not checkbox.is_selected(): # 만약 KCI 등재 체크박스가 체크되어 있지 않다면 체크
    checkbox.click()

driver.implicitly_wait(100) # 페이지 로딩이 완료될 때까지 대기

cnt = 0 
page = []

for n in range(1,187): # KCI에 등재된 논문은 총 3724건(한 페이지당 20건씩 있음)
    driver.execute_script("onclick=setPageNum({})".format(n))
    # 자바스크립트로 페이지가 이동되기 때문에 해당 코드를 실행
    time.sleep(5) # 페이지가 완전히 로드될 때까지 10초 대기
    html = driver.page_source 

    soup = BeautifulSoup(html, 'lxml') # 페이지 소스를 lxml로 변환
    # lxml : String형식의 HTML문서 -> 살아있는 HTML문서

    for_thesis = soup.find('section',attrs={"id":"searchResultList"}) # 논문 목록

    for i in for_thesis.find_all('article',attrs={'class':'thesis__summary'}):
        title = i.find('h2').get_text() # 제목 
        date = i.find('span',attrs={'class':'thesis__item'}).get_text() # 날짜
        abstract = i.find('span',attrs={'class':'thesis__abstract'}) # abstract 
        abstract2 = [abstract.get_text() if abstract != None else ''][0] 
        # 만약 abstract가 존재하지 않는다면 ''으로 대신함
        if date[:4] in [str(j) for j in range(2014,2024)]: # 2014~2023년에 해당되는 논문만(1430건)
            res += [[title, date, abstract2]] # [제목, 날짜, abstract 내용]을 리스트에 저장함
            cnt += 1

driver.close()